# Multiview 

In [1]:
import os
import glob
from pathlib import Path
import sys
from tqdm.auto import tqdm
import json
from copy import deepcopy

import pandas as pd
import numpy as np
import os
import math
import random

import torch
import torch.nn as nn
from torchvision import transforms

from PIL import Image
import cv2

from transformers import AutoProcessor, AutoImageProcessor, AutoModel, Siglip2Model, Siglip2ImageProcessor, SiglipModel, SiglipImageProcessor
from sklearn.model_selection import KFold, GroupKFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, BayesianRidge
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor, ExtraTreesRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.svm import LinearSVR
from sklearn.dummy import DummyRegressor

from sklearn.decomposition import PCA
from sklearn import model_selection
from sklearn import preprocessing

from dataclasses import dataclass
from typing import Optional, Dict

import matplotlib.pyplot as plt

2025-12-12 16:13:56.857997: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765556037.144717      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765556037.230072      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
def seeding(SEED):
    np.random.seed(SEED)
    random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    torch.manual_seed(SEED)
    # pl.seed_everything(SEED)
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(SEED)
        torch.cuda.manual_seed_all(SEED)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    print('seeding done!!!')

def flush():
    import gc
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

seeding(42)

seeding done!!!


In [3]:
@dataclass
class Config:
    # Data paths
    DATA_PATH: Path = Path("/kaggle/input/csiro-biomass/")
    TRAIN_DATA_PATH: Path = DATA_PATH/'train'
    TEST_DATA_PATH: Path = DATA_PATH/'test'

    device = "cuda" if torch.cuda.is_available() else "cpu"
    seed = 42

cfg = Config()
seeding(cfg.seed)

seeding done!!!


In [4]:
def pivot_table(df: pd.DataFrame)->pd.DataFrame:

    if 'target' in df.columns.tolist():
        df_pt = pd.pivot_table(
            df, 
            values='target', 
            index=['image_path', 'Sampling_Date', 'State', 'Species', 'Pre_GSHH_NDVI', 'Height_Ave_cm'], 
            columns='target_name', 
            aggfunc='mean'
        ).reset_index()
    else:
        df['target'] = 0
        df_pt = pd.pivot_table(
            df, 
            values='target', 
            index='image_path', 
            columns='target_name', 
            aggfunc='mean'
        ).reset_index()
    return df_pt

train_df = pd.read_csv(cfg.DATA_PATH/'train.csv')
test_df = pd.read_csv(cfg.DATA_PATH/'test.csv')
train_df = pivot_table(df=train_df)
test_df = pivot_table(df=test_df)

train_df['image_path'] = train_df['image_path'].apply(lambda p: str(cfg.DATA_PATH / p))
test_df['image_path'] = test_df['image_path'].apply(lambda p: str(cfg.DATA_PATH / p))
test_df.head()

target_name,image_path,Dry_Clover_g,Dry_Dead_g,Dry_Green_g,Dry_Total_g,GDM_g
0,/kaggle/input/csiro-biomass/test/ID1001187975.jpg,0.0,0.0,0.0,0.0,0.0


In [5]:
def melt_table(df: pd.DataFrame) -> pd.DataFrame:
    TARGET_NAMES = ['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'Dry_Total_g', 'GDM_g']
    melted = df.melt(
        id_vars='image_path',
        value_vars=TARGET_NAMES,
        var_name='target_name',
        value_name='target'
    )
    melted['sample_id'] = (
        melted['image_path']
        .str.replace(r'^.*/', '', regex=True)  # remove folder path, keep filename
        .str.replace('.jpg', '', regex=False)  # remove extension
        + '__' + melted['target_name']
    )
    
    return melted[['sample_id', 'image_path', 'target_name', 'target']]

t1 = melt_table(test_df)
t1

,sample_id,image_path,target_name,target
0,ID1001187975__Dry_Clover_g,/kaggle/input/csiro-biomass/test/ID1001187975.jpg,Dry_Clover_g,0.0
1,ID1001187975__Dry_Dead_g,/kaggle/input/csiro-biomass/test/ID1001187975.jpg,Dry_Dead_g,0.0
2,ID1001187975__Dry_Green_g,/kaggle/input/csiro-biomass/test/ID1001187975.jpg,Dry_Green_g,0.0
3,ID1001187975__Dry_Total_g,/kaggle/input/csiro-biomass/test/ID1001187975.jpg,Dry_Total_g,0.0
4,ID1001187975__GDM_g,/kaggle/input/csiro-biomass/test/ID1001187975.jpg,GDM_g,0.0


In [6]:
train_df.head(1)
train_df['Species'].value_counts()

Species
Ryegrass_Clover                                                98
Ryegrass                                                       62
Phalaris_Clover                                                42
Clover                                                         41
Fescue                                                         28
Lucerne                                                        22
Phalaris_BarleyGrass_SilverGrass_SpearGrass_Clover_Capeweed    11
Fescue_CrumbWeed                                               10
WhiteClover                                                    10
Phalaris_Ryegrass_Clover                                        8
Phalaris                                                        8
Phalaris_Clover_Ryegrass_Barleygrass_Bromegrass                 7
SubcloverLosa                                                   5
SubcloverDalkeith                                               3
Mixed                                                           2
Na

In [7]:
# from sklearn.cluster import KMeans
# from sklearn.model_selection import StratifiedGroupKFold

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.model_selection import StratifiedGroupKFold

def create_robust_cv(train_df, 
                     target_names, 
                     weights, 
                     n_splits=5, 
                     n_clusters=50, 
                     seed=42):
    """
    Creates a robust CV scheme that simulates unseen test environments.
    """
    df = train_df.copy()

    # ----------------------------
    # 1. Select Embedding Columns
    # ----------------------------
    # Robust selector for "emb", "embedding", or numeric columns
    emb_cols = [c for c in df.columns if "emb" in str(c).lower()]
    if not emb_cols:
        exclude = set(target_names + list(weights.keys()) + ['fold', 'id', 'image_path', 'State', 'Species'])
        emb_cols = [c for c in df.columns if c not in exclude and str(c).isdigit()]
    
    print(f"Using {len(emb_cols)} embedding columns for clustering.")

    # ----------------------------
    # 2. Visual Clustering (Define the Groups)
    # ----------------------------
    # We create ~50 groups. The model must learn to generalize *across* groups.
    X_emb = df[emb_cols].values
    kmeans = KMeans(n_clusters=n_clusters, random_state=seed, n_init=10)
    df["visual_cluster"] = kmeans.fit_predict(X_emb)

    # ----------------------------
    # 3. Create Stratification Target
    # ----------------------------
    # We want folds to be balanced by BIOMASS, not by cluster ID.
    composite = np.zeros(len(df))
    for t in target_names:
        composite += df[t] * weights.get(t, 0)
        
    # Bin the continuous target into 10 bins for stratification
    try:
        df['target_bins'] = pd.qcut(composite, q=10, labels=False, duplicates='drop')
    except ValueError:
        df['target_bins'] = pd.qcut(composite, q=5, labels=False, duplicates='drop')

    # ----------------------------
    # 4. Stratified Group K-Fold
    # ----------------------------
    # CRITICAL: 
    #   y      = target_bins    (Balance the biomass difficulty)
    #   groups = visual_cluster (Prevent leakage of similar images)
    sgkf = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state=seed)

    df["fold"] = -1
    for fold, (train_idx, val_idx) in enumerate(sgkf.split(df, df['target_bins'], groups=df["visual_cluster"])):
        df.loc[val_idx, "fold"] = fold

    return df

# ==========================================
# CONFIGURATION
# ==========================================
TARGET_NAMES = ['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'Dry_Total_g', 'GDM_g']
weights = {
    'Dry_Green_g': 0.1, 'Dry_Dead_g': 0.1, 'Dry_Clover_g': 0.1,
    'GDM_g': 0.2, 'Dry_Total_g': 0.5,
}

# LOAD AND CREATE FOLDS
train_df = pd.read_csv("/kaggle/input/csiro-image-embeddings/train_siglip_embeddings.csv")
train_df = create_robust_cv(train_df, TARGET_NAMES, weights, n_splits=5, n_clusters=50)

# VERIFICATION
print("\nFold Distribution (Will be uneven - this is expected):")
print(train_df['fold'].value_counts())

print("\nVerify Leakage (Must be 0):")
leakage_check = train_df.groupby('visual_cluster')['fold'].nunique()
print(f"Clusters appearing in multiple folds: {sum(leakage_check > 1)}")

Using 1152 embedding columns for clustering.

Fold Distribution (Will be uneven - this is expected):
fold
0    83
2    74
3    74
1    68
4    58
Name: count, dtype: int64

Verify Leakage (Must be 0):
Clusters appearing in multiple folds: 0


In [8]:
def split_image(image, patch_size=520, overlap=16):
    h, w, c = image.shape
    stride = patch_size - overlap
    
    patches = []
    coords  = []   # (y1, x1, y2, x2)
    
    for y in range(0, h, stride):
        for x in range(0, w, stride):
            y1 = y
            x1 = x
            y2 = y + patch_size
            x2 = x + patch_size
            
            # Pad last patch if needed (very rare with your fixed 1000×2000)
            patch = image[y1:y2, x1:x2, :]
            if patch.shape[0] < patch_size or patch.shape[1] < patch_size:
                pad_h = patch_size - patch.shape[0]
                pad_w = patch_size - patch.shape[1]
                patch = np.pad(patch, ((0,pad_h), (0,pad_w), (0,0)), mode='reflect')
            
            patches.append(patch)
            coords.append((y1, x1, y2, x2))
    
    return patches, coords

In [9]:
def get_model(model_path: str, device: str = 'cpu'):
    model = AutoModel.from_pretrained(
        model_path,
        local_files_only=True
    )
    processor = AutoImageProcessor.from_pretrained(model_path)
    return model.eval().to(device), processor

dino_path = "/kaggle/input/dinov2/pytorch/giant/1"
siglip_path = "/kaggle/input/google-siglip-so400m-patch14-384/transformers/default/1"

# dino_model, dino_processor = get_model(
#     model_path=dino_path, device=device
# )

# siglip_model, siglip_processor = get_model(
#     model_path=siglip_path, device=device
# )

In [10]:
def compute_embeddings(model_path, df, patch_size=520):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(device)

    model, processor = get_model(
        model_path=model_path, device=device
    )

    IMAGE_PATHS = []
    EMBEDDINGS = []

    for i, row in tqdm(df.iterrows(), total=len(df)):
        img_path = row['image_path']
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        patches, coords = split_image(img, patch_size=patch_size)
        images = [Image.fromarray(p).convert("RGB") for p in patches]

        inputs = processor(images=images, return_tensors="pt").to(model.device)
        with torch.no_grad():
            if 'siglip' in model_path:
                features = model.get_image_features(**inputs)
            elif 'dino' in model_path:
                features = model(**inputs).pooler_output
                # patches = model(**inputs).last_hidden_state
                # features = patches[:, 0, :]
            else:
                raise Exception("Model should be dino or siglip")
        embeds = features.mean(dim=0).detach().cpu().numpy()
        EMBEDDINGS.append(embeds)
        IMAGE_PATHS.append(img_path)

    embeddings = np.stack(EMBEDDINGS, axis=0)
    n_features = embeddings.shape[1]
    emb_columns = [f"emb{i+1}" for i in range(n_features)]
    emb_df = pd.DataFrame(embeddings, columns=emb_columns)
    emb_df['image_path'] = IMAGE_PATHS
    df_final = df.merge(emb_df, on='image_path', how='left')
    flush()
    return df_final 

# train_siglip_df = compute_embeddings(model_path=siglip_path, df=train_df, patch_size=520)
train_siglip_df = train_df.copy()
test_siglip_df = compute_embeddings(model_path=siglip_path, df=test_df, patch_size=520)

flush()

cpu


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

  0%|          | 0/1 [00:00<?, ?it/s]

# Text embeddings

In [11]:
# "dense pasture grass",
#         "sparse pasture vegetation",
#         "patchy grass cover",
#         "bare soil patches in grass",
#         "thick tangled grass",
#         "open low-density pasture",
#         "dry cracked soil",
#         "dry canopy",
#         "low moisture vegetation",
#         "dry pasture with yellow tones",
#         "wilted grass"

In [48]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoModel, AutoTokenizer

# Your specific path
SIGLIP_PATH = "/kaggle/input/google-siglip-so400m-patch14-384/transformers/default/1"

def generate_semantic_features(image_embeddings, model_path=SIGLIP_PATH):
    """
    Projects image embeddings onto agronomic text concepts.
    Returns: (N_samples, N_prompts) numpy array
    """
    print(f"Loading SigLIP Text Encoder from {model_path}...")

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    # 1. Load Model (Text part only ideally, but SigLIP usually loads all)
    try:
        model = AutoModel.from_pretrained(model_path).to(device)
        tokenizer = AutoTokenizer.from_pretrained(model_path)
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

    # 2. Define Agronomic Prompts
    # These act as "Virtual Sensors" for specific pasture traits
    prompts = [
        "dense pasture grass",
        "open low-density pasture",
        "sparse pasture vegetation",
        "patchy grass cover",
        "bare soil patches",
        "thick tangled grass",
        "overgrazed pasture",
        "ungrazed mature pasture",
        "pasture with accumulated dead material",
        "standing dead biomass",
        "tall grass",
        "short grass",
        "overgrown grass",
        "recently grazed pasture",
        "clover pasture",
        "white clover pasture",
        "subclover pasture",
        "lucerne pasture",
        "ryegrass pasture",
        "phalaris pasture",
        "fescue pasture",
        "mixed pasture grasses",
        "ryegrass and clover mix",
        "phalaris and clover pasture",
        "ryegrass clover phalaris mix"
    ]
    
    # 3. Encode Text
    print("Encoding prompts...")
    text_inputs = tokenizer(prompts, padding="max_length", return_tensors="pt").to(device)
    with torch.no_grad():
        text_emb = model.get_text_features(**text_inputs)
        # Normalize text embeddings
        text_emb = text_emb / text_emb.norm(p=2, dim=-1, keepdim=True)
    
    # 4. Compute Similarity (Dot Product)
    # Ensure image_embeddings is a torch tensor on the right device
    if isinstance(image_embeddings, np.ndarray):
        img_tensor = torch.tensor(image_embeddings, dtype=torch.float32).to(device)
    else:
        img_tensor = image_embeddings.to(device)
        
    # Normalize image embeddings
    img_tensor = img_tensor / img_tensor.norm(p=2, dim=-1, keepdim=True)
    
    # Matrix Multiplication: (N_imgs, 768) @ (768, N_prompts) -> (N_imgs, N_prompts)
    semantic_scores = torch.matmul(img_tensor, text_emb.T).cpu().numpy()
    
    # Clean up
    del model, text_inputs, text_emb, img_tensor
    torch.cuda.empty_cache()
    
    return semantic_scores

# --- USAGE ---
# Assuming 'embeddings_all' is your full (N, 768) array of SigLIP image embeddings
# If you have train/test split, concat them first to generate features for all
# semantic_feats_all = generate_semantic_features(embeddings_all)

# Feature engineering

In [49]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics.pairwise import cosine_similarity

class SupervisedEmbeddingEngine(BaseEstimator, TransformerMixin):
    def __init__(self, 
                 n_pca=0.95, 
                 n_pls=8,
                 n_gmm=5,
                 random_state=42):
        
        self.n_pca = n_pca
        self.n_pls = n_pls
        self.n_gmm = n_gmm
        self.random_state = random_state
        
        # Transformers
        self.scaler = StandardScaler()
        self.pca = PCA(n_components=n_pca, random_state=random_state)
        self.pls = PLSRegression(n_components=n_pls, scale=False)
        self.gmm = GaussianMixture(n_components=n_gmm, covariance_type='diag', random_state=random_state)

        # Internal State
        self.global_mean_ = None
        self.pls_fitted_ = False

    def fit(self, X, y=None, X_semantic=None):
        """
        X: Raw Embeddings (N, D)
        y: Targets
        X_semantic: (Optional) Semantic Scores (N, K)
        """
        
        # 1. Scale Raw Embeddings
        X_scaled = self.scaler.fit_transform(X)
        self.global_mean_ = np.mean(X_scaled, axis=0)
        
        # 2. Fit Unsupervised transformers
        self.pca.fit(X_scaled)
        self.gmm.fit(X_scaled)
        
        # 3. Fit PLS if y is provided
        if y is not None:
            y_clean = y.values if hasattr(y, 'values') else y
            if y_clean.ndim == 1:
                y_clean = y_clean.reshape(-1, 1)

            self.pls.fit(X_scaled, y_clean)
            self.pls_fitted_ = True
        
        return self

    def transform(self, X, X_semantic=None):
        X_scaled = self.scaler.transform(X)
        return self._generate_features(X_scaled, X_semantic)

    def _generate_features(self, X_scaled, X_semantic=None):
        features = []
        
        # A. Optional Semantic Features
        if X_semantic is not None:
            features.append(X_semantic)
            if X_semantic.shape[1] >= 2:
                interact = (X_semantic[:, 0] * X_semantic[:, 2]).reshape(-1, 1)
                features.append(interact)

        # B. PCA components
        f_pca = self.pca.transform(X_scaled)
        features.append(f_pca)
        
        # C. PLS components
        if self.pls_fitted_:
            f_pls = self.pls.transform(X_scaled)
            features.append(f_pls)
        
        # D. GMM cluster probabilities
        f_gmm = self.gmm.predict_proba(X_scaled)
        features.append(f_gmm)
        
        # E. Norm + Typicality
        f_mag = np.linalg.norm(X_scaled, axis=1, keepdims=True)
        f_sim = cosine_similarity(X_scaled, self.global_mean_.reshape(1, -1))
        features.extend([f_mag, f_sim])
        
        return np.hstack(features)

In [50]:
COLUMNS = train_df.filter(like="emb").columns
TARGET_NAMES = ['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'Dry_Total_g', 'GDM_g']
weights = {
    'Dry_Green_g': 0.1,
    'Dry_Dead_g': 0.1,
    'Dry_Clover_g': 0.1,
    'GDM_g': 0.2,
    'Dry_Total_g': 0.5,
}

TARGET_MAX = {
    "Dry_Clover_g": 71.7865,
    "Dry_Dead_g": 83.8407,
    "Dry_Green_g": 157.9836,
    "Dry_Total_g": 185.70,
    "GDM_g": 157.9836,
}

def competition_metric(y_true, y_pred) -> float:
    y_weighted = 0
    for l, label in enumerate(TARGET_NAMES):
        y_weighted = y_weighted + y_true[:, l].mean() * weights[label]

    ss_res = 0
    ss_tot = 0
    for l, label in enumerate(TARGET_NAMES):
        ss_res = ss_res + ((y_true[:, l] - y_pred[:, l])**2).mean() * weights[label]
        ss_tot = ss_tot + ((y_true[:, l] - y_weighted)**2).mean() * weights[label]

    return 1 - ss_res / ss_tot

In [51]:
# def post_process_biomass(df_preds):
#     """
#     Enforces physical mass balance constraints on biomass predictions.
    
#     Constraints enforced:
#     1. Dry_Green_g + Dry_Clover_g = GDM_g
#     2. GDM_g + Dry_Dead_g = Dry_Total_g
    
#     Method:
#     Uses Orthogonal Projection. It finds the set of values that satisfy
#     the constraints while minimizing the Euclidean distance to the 
#     original model predictions.
    
#     Args:
#         df_preds (pd.DataFrame): DataFrame containing the 5 prediction columns.
        
#     Returns:
#         pd.DataFrame: A new DataFrame with consistent, non-negative values.
#     """
#     # 1. Define the specific order required for the math
#     # We treat the vector x as: [Green, Clover, Dead, GDM, Total]
#     ordered_cols = [
#         "Dry_Green_g", 
#         "Dry_Clover_g", 
#         "Dry_Dead_g", 
#         "GDM_g", 
#         "Dry_Total_g"
#     ]
    
#     # Check if columns exist
#     if not all(col in df_preds.columns for col in ordered_cols):
#         missing = [c for c in ordered_cols if c not in df_preds.columns]
#         raise ValueError(f"Input DataFrame is missing columns: {missing}")

#     # 2. Extract values in the specific order -> Shape (N_samples, 5)
#     Y = df_preds[ordered_cols].values.T  # Transpose to (5, N) for matrix math

#     # 3. Define the Constraint Matrix C
#     # We want Cx = 0
#     # Eq 1: 1*Green + 1*Clover + 0*Dead - 1*GDM + 0*Total = 0
#     # Eq 2: 0*Green + 0*Clover + 1*Dead + 1*GDM - 1*Total = 0
#     C = np.array([
#         [1, 1, 0, -1,  0],
#         [0, 0, 1,  1, -1]
#     ])

#     # 4. Calculate Projection Matrix P
#     # P = I - C^T * (C * C^T)^-1 * C
#     # This projects any vector onto the null space of C (the valid subspace)
#     C_T = C.T
#     inv_CCt = np.linalg.inv(C @ C_T)
#     P = np.eye(5) - C_T @ inv_CCt @ C

#     # 5. Apply Projection
#     # Y_new = P * Y
#     Y_reconciled = P @ Y

#     # 6. Transpose back to (N, 5)
#     Y_reconciled = Y_reconciled.T

#     # 7. Post-correction for negatives
#     # Projection can mathematically create negative values (e.g. if Total was predicted 0)
#     # We clip to 0. Note: This might slightly break the sum equality again, 
#     # but exact equality with negatives is physically impossible anyway.
#     Y_reconciled = Y_reconciled.clip(min=0)

#     # 8. Create Output DataFrame
#     df_out = df_preds.copy()
#     df_out[ordered_cols] = Y_reconciled

#     return df_out

def post_process_biomass(df_preds):
    """
    Enforces mass balance constraints hierarchically.
    
    Philosophy: 
    - GDM_g is trusted. Green/Clover are scaled to match it.
    - Dry_Total_g is trusted. Dead is derived as (Total - GDM).
    - If constraints are physically impossible (e.g. GDM > Total),
      we assume Total was underestimated and raise it to match GDM.
      
    Args:
        df_preds (pd.DataFrame): Predictions
        
    Returns:
        pd.DataFrame: Consistently processed dataframe.
    """
    # Create a copy to avoid SettingWithCopy warnings
    df_out = df_preds.copy()
    
    # ---------------------------------------------------------
    # 1. Enforce: Dry_Green_g + Dry_Clover_g = GDM_g
    # ---------------------------------------------------------
    # We trust the *magnitude* of GDM_g more than the components.
    # We trust the *ratio* of Green vs Clover from the model.
    
    # Calculate current component sum
    comp_sum = df_out["Dry_Green_g"] + df_out["Dry_Clover_g"]
    
    # Avoid division by zero
    mask_nonzero = comp_sum > 1e-9
    
    # Calculate scaling factor so components sum exactly to GDM
    scale_factor = df_out.loc[mask_nonzero, "GDM_g"] / comp_sum[mask_nonzero]
    
    # Apply scaling
    df_out.loc[mask_nonzero, "Dry_Green_g"] *= scale_factor
    df_out.loc[mask_nonzero, "Dry_Clover_g"] *= scale_factor
    
    # Edge case: If comp_sum is 0 but GDM is not, we can't scale.
    # (Optional: You could split GDM evenly, but usually the model predicts 0 GDM here too)
    
    # ---------------------------------------------------------
    # 2. Enforce: GDM_g + Dry_Dead_g = Dry_Total_g
    # ---------------------------------------------------------
    # You stated Dead is hard to predict. 
    # Therefore, we discard the direct prediction of Dead and derive it.
    
    df_out["Dry_Dead_g"] = df_out["Dry_Total_g"] - df_out["GDM_g"]
    
    # ---------------------------------------------------------
    # 3. Handle Physical Impossibilities (Negative Dead)
    # ---------------------------------------------------------
    # If Dead < 0, it means GDM > Total. This is physically impossible.
    # Logic: GDM is a sum of living parts (robust). Total is the scan of everything.
    # If GDM > Total, the model likely underestimated Total.
    
    neg_dead_mask = df_out["Dry_Dead_g"] < 0
    
    if neg_dead_mask.any():
        # Set Dead to 0 (cannot be negative)
        df_out.loc[neg_dead_mask, "Dry_Dead_g"] = 0
        
        # Raise Total to match GDM (maintaining balance)
        df_out.loc[neg_dead_mask, "Dry_Total_g"] = df_out.loc[neg_dead_mask, "GDM_g"]

    return df_out

def compare_results(oof, train_data):
    y_oof_df = pd.DataFrame(oof, columns=TARGET_NAMES) # ensure columns match
    # 2. Check Score BEFORE Processing
    raw_score = competition_metric(train_data[TARGET_NAMES].values, y_oof_df.values)
    print(f"Raw CV Score: {raw_score:.6f}")
    
    # 3. Apply Post-Processing
    y_oof_proc = post_process_biomass(y_oof_df)
    
    # 4. Check Score AFTER Processing
    proc_score = competition_metric(train_data[TARGET_NAMES].values, y_oof_proc.values)
    print(f"Processed CV Score: {proc_score:.6f}")
    
    print(f"Improvement: {raw_score - proc_score:.6f}")

In [52]:
# from sklearn.preprocessing import PowerTransformer, QuantileTransformer, RobustScaler
# from sklearn.svm import SVR

# def cross_validate(model, train_data, test_data, feature_engine, target_transform='max', n_splits=5, seed=42):
#     """
#     target_transform options:
#     - 'max': Linear scaling by max value (Preserves distribution shape)
#     - 'log': np.log1p (Aggressive compression of outliers)
#     - 'sqrt': np.sqrt (Moderate compression, good for biological counts/area)
#     - 'yeo-johnson': PowerTransformer (Makes data Gaussian-like automatically)
#     - 'quantile': QuantileTransformer (Forces strict Normal distribution)
#     """
    
#     target_max_arr = np.array([TARGET_MAX[t] for t in TARGET_NAMES], dtype=float)
#     y_true = train_data[TARGET_NAMES]
    
#     y_pred = pd.DataFrame(0.0, index=train_data.index, columns=TARGET_NAMES)
#     y_pred_test = np.zeros([len(test_data), len(TARGET_NAMES)], dtype=float)

#     for fold in range(n_splits):
#         seeding(seed*(seed//2 + fold))
#         # 1. Split Data
#         train_mask = train_data['fold'] != fold
#         valid_mask = train_data['fold'] == fold
#         val_idx = train_data[valid_mask].index

#         X_train_raw = train_data[train_mask][COLUMNS].values
#         X_valid_raw = train_data[valid_mask][COLUMNS].values
#         X_test_raw = test_data[COLUMNS].values
        
#         y_train = train_data[train_mask][TARGET_NAMES].values
#         y_valid = train_data[valid_mask][TARGET_NAMES].values

#         # ===========================
#         # 2) TARGET TRANSFORMATION
#         # ===========================
#         transformer = None # To store stateful transformers (Yeo/Quantile)
        
#         if target_transform == 'log':
#             y_train_proc = np.log1p(y_train)
            
#         elif target_transform == 'max':
#             y_train_proc = y_train / target_max_arr
            
#         elif target_transform == 'sqrt':
#             # Great for biomass/area data (Variance stabilizing)
#             y_train_proc = np.sqrt(y_train)
            
#         elif target_transform == 'yeo-johnson':
#             # Learns optimal parameter to make data Gaussian
#             transformer = PowerTransformer(method='yeo-johnson', standardize=True)
#             y_train_proc = transformer.fit_transform(y_train)
            
#         elif target_transform == 'quantile':
#             # Forces data into a normal distribution (Robust to outliers)
#             # transformer = QuantileTransformer(output_distribution='uniform', n_quantiles=64, random_state=42)
#             transformer = RobustScaler()
#             y_train_proc = transformer.fit_transform(y_train)
            
#         else:
#             y_train_proc = y_train

#         # ==========================================
#         # 3) FEATURE ENGINEERING
#         # ==========================================
#         engine = deepcopy(feature_engine)
#         # Note: If your engine uses PLS, pass the transformed y!
#         engine.fit(X_train_raw, y=y_train_proc) 
        
#         x_train_eng = engine.transform(X_train_raw)
#         x_valid_eng = engine.transform(X_valid_raw)
#         x_test_eng = engine.transform(X_test_raw)
        
#         # ==========================================
#         # 4) TRAIN & PREDICT
#         # ==========================================
#         fold_valid_pred = np.zeros_like(y_valid)
#         fold_test_pred = np.zeros([len(test_data), len(TARGET_NAMES)])

#         for k in range(len(TARGET_NAMES)):
#             regr = deepcopy(model)
#             regr.fit(x_train_eng, y_train_proc[:, k])
            
#             # Raw Predictions (in transformed space)
#             pred_valid_raw = regr.predict(x_valid_eng)
#             pred_test_raw = regr.predict(x_test_eng)
            
#             # Store raw for inverse transform block below
#             fold_valid_pred[:, k] = pred_valid_raw
#             fold_test_pred[:, k] = pred_test_raw

#         # ===========================
#         # 5) INVERSE TRANSFORM (Apply to full matrix)
#         # ===========================
#         if target_transform == 'log':
#             fold_valid_pred = np.expm1(fold_valid_pred)
#             fold_test_pred = np.expm1(fold_test_pred)
            
#         elif target_transform == 'max':
#             fold_valid_pred = fold_valid_pred * target_max_arr
#             fold_test_pred = fold_test_pred * target_max_arr
            
#         elif target_transform == 'sqrt':
#             # Inverse of sqrt is square
#             fold_valid_pred = np.square(fold_valid_pred)
#             fold_test_pred = np.square(fold_test_pred)
            
#         elif target_transform in ['yeo-johnson', 'quantile']:
#             # Use the fitted transformer to invert
#             fold_valid_pred = transformer.inverse_transform(fold_valid_pred)
#             fold_test_pred = transformer.inverse_transform(fold_test_pred)

#         # # Final Clip (Biomass cannot be negative)
#         # fold_valid_pred = fold_valid_pred.clip(min=0)
#         # fold_test_pred = fold_test_pred.clip(min=0)

#         # Store results
#         y_pred.loc[val_idx] = fold_valid_pred
#         y_pred_test += fold_test_pred / n_splits
        
#         if fold == 0:
#             print(f"  [Fold 0] Target: {target_transform}, Feats: {x_train_eng.shape}")

#     full_cv = competition_metric(y_true.values, y_pred.values)
#     print(f"Full CV Score: {full_cv:.6f}")
    
#     return y_pred.values, y_pred_test

# # Initialize
# feat_engine = SupervisedEmbeddingEngine(
#     n_pca=0.96,
#     n_pls=8,             # Extract 8 strong supervised signals
#     n_gmm=6,             # 6 Soft clusters
# )

# print("######## Ridge Regression #######")
# oof_ridge, pred_test_ri = cross_validate(Ridge(), train_siglip_df, test_siglip_df, feature_engine=feat_engine)
# compare_results(oof_ridge, train_siglip_df)

# print("####### Lasso Regression #######")
# oof_la, pred_test_la = cross_validate(Lasso(), train_siglip_df, test_siglip_df, feature_engine=feat_engine)
# compare_results(oof_la, train_siglip_df)

# print("\n###### GradientBoosting Regressor #######")
# oof_gb, pred_test_gb = cross_validate(GradientBoostingRegressor(), train_siglip_df, test_siglip_df, feature_engine=feat_engine, target_transform='max')
# compare_results(oof_gb, train_siglip_df)

# print("\n###### Hist Gradient Boosting Regressor ######")
# oof_hb, pred_test_hb = cross_validate(HistGradientBoostingRegressor(), train_siglip_df, test_siglip_df, feature_engine=feat_engine, target_transform='max')
# compare_results(oof_hb, train_siglip_df)

# print("\n##### CAT Regressor ######")
# oof_cat, pred_test_cat = cross_validate(
#     CatBoostRegressor(verbose=0), 
#     train_siglip_df, test_siglip_df, 
#     feature_engine=feat_engine
# )
# compare_results(oof_cat, train_siglip_df)

# print("\n######## XGB #######")
# oof_xgb, pred_test_xgb = cross_validate(
#     XGBRegressor(verbosity=0), 
#     train_siglip_df, test_siglip_df, 
#     feature_engine=feat_engine, 
#     target_transform='max')
# compare_results(oof_xgb, train_siglip_df)

# print("\n######## LGBM #######")
# oof_lgbm, pred_test_lgbm = cross_validate(
#     LGBMRegressor(verbose=-1), 
#     train_siglip_df, test_siglip_df, 
#     feature_engine=feat_engine, 
#     target_transform='max')
# compare_results(oof_lgbm, train_siglip_df)

## Semantic probing

In [53]:
# --- STEP 1: Generate Semantic Features ---
# We combine Train and Test to generate features in one go, then split them back.
# This ensures the text-projections are consistent.

# Concatenate embeddings
X_all_emb = np.vstack([
    train_siglip_df[COLUMNS].values, 
    test_siglip_df[COLUMNS].values
])

# Generate Semantic Probes (Using the function defined in Part 1)
# Make sure SIGLIP_PATH is correct for your environment
print("Generating Semantic Features via SigLIP Text Encoder...")
try:
    all_semantic_scores = generate_semantic_features(X_all_emb, model_path=SIGLIP_PATH)
    
    # Split back into Train and Test
    n_train = len(train_siglip_df)
    sem_train_full = all_semantic_scores[:n_train]
    sem_test_full = all_semantic_scores[n_train:]
    print(f"Semantic Features Generated. Train: {sem_train_full.shape}, Test: {sem_test_full.shape}")
    
except Exception as e:
    print(f"Skipping Semantic Features due to error: {e}")
    # Fallback to None if model path is wrong or memory fails
    sem_train_full = None
    sem_test_full = None

Generating Semantic Features via SigLIP Text Encoder...
Loading SigLIP Text Encoder from /kaggle/input/google-siglip-so400m-patch14-384/transformers/default/1...
Encoding prompts...
Semantic Features Generated. Train: (357, 25), Test: (1, 25)


In [54]:
# --- STEP 2: Updated Cross-Validation Function ---
def cross_validate(model, train_data, test_data, feature_engine, 
                   semantic_train=None, semantic_test=None, # <--- NEW ARGS
                   target_transform='max', n_splits=5, seed=42):
    
    # Setup Targets
    target_max_arr = np.array([TARGET_MAX[t] for t in TARGET_NAMES], dtype=float)
    y_true = train_data[TARGET_NAMES]
    
    # Setup Storage
    y_pred = pd.DataFrame(0.0, index=train_data.index, columns=TARGET_NAMES)
    y_pred_test = np.zeros([len(test_data), len(TARGET_NAMES)], dtype=float)

    for fold in range(n_splits):
        seeding(seed*(seed//2 + fold))
        # Create masks
        train_mask = train_data['fold'] != fold
        valid_mask = train_data['fold'] == fold
        val_idx = train_data[valid_mask].index

        # Raw Inputs (Embeddings)
        X_train_raw = train_data[train_mask][COLUMNS].values
        X_valid_raw = train_data[valid_mask][COLUMNS].values
        X_test_raw = test_data[COLUMNS].values
        
        # Semantic Inputs (Slicing)
        # We handle the case where semantic features might be None
        sem_train_fold = semantic_train[train_mask] if semantic_train is not None else None
        sem_valid_fold = semantic_train[valid_mask] if semantic_train is not None else None
        
        # Raw Targets
        y_train = train_data[train_mask][TARGET_NAMES].values
        y_valid = train_data[valid_mask][TARGET_NAMES].values

        # ===========================
        # 1) TRANSFORM TARGETS
        # ===========================
        if target_transform == 'log':
            y_train_proc = np.log1p(y_train)
        elif target_transform == 'max':
            y_train_proc = y_train / target_max_arr
        else:
            y_train_proc = y_train

        # ==========================================
        # 2) FEATURE ENGINEERING
        # ==========================================
        engine = deepcopy(feature_engine)
        
        # FIT: Now passes y (for PLS/RFE) and Semantic Features
        engine.fit(X_train_raw, y=y_train_proc, X_semantic=sem_train_fold)
        
        # TRANSFORM: Pass Semantic Features
        x_train_eng = engine.transform(X_train_raw, X_semantic=sem_train_fold)
        x_valid_eng = engine.transform(X_valid_raw, X_semantic=sem_valid_fold)
        # For test, we use the full test semantic set
        x_test_eng = engine.transform(X_test_raw, X_semantic=semantic_test)
        
        # ==========================================
        # 3) TRAIN & PREDICT
        # ==========================================
        fold_valid_pred = np.zeros_like(y_valid)
        fold_test_pred = np.zeros([len(test_data), len(TARGET_NAMES)])

        for k in range(len(TARGET_NAMES)):
            regr = deepcopy(model)
            
            # Fit model
            regr.fit(x_train_eng, y_train_proc[:, k])
            
            # Predict
            pred_valid_raw = regr.predict(x_valid_eng)
            pred_test_raw = regr.predict(x_test_eng)
            
            # ===========================
            # 4) INVERSE TRANSFORM
            # ===========================
            if target_transform == 'log':
                pred_valid_inv = np.expm1(pred_valid_raw)
                pred_test_inv = np.expm1(pred_test_raw)
            elif target_transform == 'max':
                pred_valid_inv = (pred_valid_raw * target_max_arr[k])
                pred_test_inv = (pred_test_raw * target_max_arr[k])
            else:
                pred_valid_inv = pred_valid_raw
                pred_test_inv = pred_test_raw

            fold_valid_pred[:, k] = pred_valid_inv
            fold_test_pred[:, k] = pred_test_inv

        # Store results
        y_pred.loc[val_idx] = fold_valid_pred
        y_pred_test += fold_test_pred / n_splits
        
        if fold == 0:
            print(f"  [Fold 0 Info] Target: {target_transform}, Feats: {x_train_eng.shape}")

    full_cv = competition_metric(y_true.values, y_pred.values)
    print(f"Full CV Score: {full_cv:.6f}")
    
    return y_pred.values, y_pred_test

# --- STEP 3: Run Models ---

# Initialize the NEW Supervised Engine
feat_engine = SupervisedEmbeddingEngine(
    n_pca=0.80,
    n_pls=8,             # Supervised signals
    n_gmm=6,             # Soft clusters
)

# print("######## Ridge Regression #######")
# oof_ridge, pred_test_ri = cross_validate(
#     Ridge(), 
#     train_siglip_df, test_siglip_df, 
#     feature_engine=feat_engine,
#     semantic_train=sem_train_full, # <--- Pass Semantic
#     semantic_test=sem_test_full    # <--- Pass Semantic
# )
# compare_results(oof_ridge, train_siglip_df)

# print("\n####### Lasso Regression #######")
# # Lasso should perform much better now due to PLS and RFE
# oof_la, pred_test_la = cross_validate(
#     Lasso(alpha=0.015), # Small alpha for normalized feats
#     train_siglip_df, test_siglip_df, 
#     feature_engine=feat_engine,
#     semantic_train=sem_train_full,
#     semantic_test=sem_test_full
# )
# compare_results(oof_la, train_siglip_df)

print("\n###### GradientBoosting Regressor #######")
oof_gb, pred_test_gb = cross_validate(
    GradientBoostingRegressor(), 
    train_siglip_df, test_siglip_df, 
    feature_engine=feat_engine,
    semantic_train=sem_train_full,
    semantic_test=sem_test_full
)
compare_results(oof_gb, train_siglip_df)

print("\n###### Hist Gradient Boosting Regressor ######")
oof_hb, pred_test_hb = cross_validate(
    HistGradientBoostingRegressor(), 
    train_siglip_df, test_siglip_df, 
    feature_engine=feat_engine,
    semantic_train=sem_train_full,
    semantic_test=sem_test_full
)
compare_results(oof_hb, train_siglip_df)

print("\n##### CAT Regressor ######")
oof_cat, pred_test_cat = cross_validate(
    CatBoostRegressor(verbose=0), 
    train_siglip_df, test_siglip_df, 
    feature_engine=feat_engine,
    semantic_train=sem_train_full,
    semantic_test=sem_test_full
)
compare_results(oof_cat, train_siglip_df)

# print("\n######## XGB #######")
# oof_xgb, pred_test_xgb = cross_validate(
#     XGBRegressor(verbosity=0), 
#     train_siglip_df, test_siglip_df, 
#     feature_engine=feat_engine, 
#     semantic_train=sem_train_full,
#     semantic_test=sem_test_full,
#     target_transform='max')
# compare_results(oof_xgb, train_siglip_df)

print("\n######## LGBM #######")
oof_lgbm, pred_test_lgbm = cross_validate(
    LGBMRegressor(verbose=-1), 
    train_siglip_df, test_siglip_df, 
    feature_engine=feat_engine, 
    semantic_train=sem_train_full,
    semantic_test=sem_test_full,
    target_transform='max')
compare_results(oof_lgbm, train_siglip_df)


###### GradientBoosting Regressor #######
seeding done!!!
  [Fold 0 Info] Target: max, Feats: (274, 59)
seeding done!!!
seeding done!!!
seeding done!!!
seeding done!!!
Full CV Score: 0.687886
Raw CV Score: 0.687886
Processed CV Score: 0.691509
Improvement: -0.003624

###### Hist Gradient Boosting Regressor ######
seeding done!!!
  [Fold 0 Info] Target: max, Feats: (274, 59)
seeding done!!!
seeding done!!!
seeding done!!!
seeding done!!!
Full CV Score: 0.705588
Raw CV Score: 0.705588
Processed CV Score: 0.705878
Improvement: -0.000290

##### CAT Regressor ######
seeding done!!!
  [Fold 0 Info] Target: max, Feats: (274, 59)
seeding done!!!
seeding done!!!
seeding done!!!
seeding done!!!
Full CV Score: 0.702867
Raw CV Score: 0.702867
Processed CV Score: 0.703164
Improvement: -0.000297

######## LGBM #######
seeding done!!!
  [Fold 0 Info] Target: max, Feats: (274, 59)
seeding done!!!
seeding done!!!
seeding done!!!
seeding done!!!
Full CV Score: 0.702186
Raw CV Score: 0.702186
Processed 

# Hyperparameter tuning

In [ ]:
# import optuna
# import numpy as np
# import pandas as pd
# from sklearn.base import clone
# from sklearn.decomposition import PCA
# from sklearn.multioutput import MultiOutputRegressor
# from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor
# from catboost import CatBoostRegressor

# # ---------------------------------------------------------
# # 1. Corrected Helper Function
# # ---------------------------------------------------------
# feat_engine = EmbeddingFeatureEngine(
#     n_pca_components=0.90, 
#     n_clusters=25, 
#     use_stats=True, 
#     use_similarity=True,
#     use_anomaly=True,        # Adds Anomaly Score
#     use_entropy=True,        # Adds Entropy
#     use_pca_interactions=True # Adds Poly features on Top 5 PCA
# )

# def get_cv_score(model, train_data, feature_engine, target_transform='max', random_state=42):
#     """
#     Runs CV on ALL folds dynamically to return a single score.
#     Optimized for speed (vectorized target processing).
    
#     Args:
#         model: Estimator (must support Multi-Output or be wrapped in MultiOutputRegressor)
#         train_data: DataFrame containing 'fold' column
#         feature_engine: Transformer with .fit() and .transform()
#         target_transform: 'log', 'max', or None
#     """
#     # 1. Setup global constants
#     target_max_arr = np.array([TARGET_MAX[t] for t in TARGET_NAMES], dtype=float)
#     y_true = train_data[TARGET_NAMES].values
#     y_pred = np.zeros([len(train_data), len(TARGET_NAMES)], dtype=float)
    
#     # 2. Detect Folds dynamically
#     folds = sorted(train_data['fold'].unique())
    
#     # 3. Loop over folds
#     for fold in folds:
#         # -------------------------
#         # Data Slicing
#         # -------------------------
#         train_mask = train_data['fold'] != fold
#         valid_mask = train_data['fold'] == fold
#         val_idx = train_data[valid_mask].index

#         X_train_raw = train_data.loc[train_mask, COLUMNS].values
#         X_valid_raw = train_data.loc[valid_mask, COLUMNS].values
        
#         y_train = train_data.loc[train_mask, TARGET_NAMES].values
#         # y_valid used implicitely via y_true at the end

#         # -------------------------
#         # A) Transform Targets
#         # -------------------------
#         if target_transform == 'log':
#             y_train_proc = np.log1p(y_train)
#         elif target_transform == 'max':
#             y_train_proc = y_train / target_max_arr
#         else:
#             y_train_proc = y_train

#         # -------------------------
#         # B) Feature Engineering
#         # -------------------------
#         # Fit engine only on training split
#         engine = deepcopy(feature_engine)
#         engine.fit(X_train_raw)
        
#         X_train_eng = engine.transform(X_train_raw)
#         X_valid_eng = engine.transform(X_valid_raw)

#         # -------------------------
#         # C) Fit Model (Multi-Output)
#         # -------------------------
#         regr = clone(model)
#         regr.fit(X_train_eng, y_train_proc)

#         # -------------------------
#         # D) Predict & Inverse Transform
#         # -------------------------
#         valid_pred_raw = np.array(regr.predict(X_valid_eng))
        
#         if target_transform == 'log':
#             valid_pred = np.expm1(valid_pred_raw)
#         elif target_transform == 'max':
#             valid_pred = valid_pred_raw * target_max_arr
#         else:
#             valid_pred = valid_pred_raw

#         # Clip and Store
#         y_pred[val_idx] = valid_pred.clip(0)

#     # 4. Calculate Metric
#     score = competition_metric(y_true, y_pred)
    
#     # # Clean output buffer if running in a loop
#     # try:
#     #     from IPython.display import flush_ipython
#     #     flush_ipython()
#     # except ImportError:
#     #     pass
        
#     return score

# # ---------------------------------------------------------
# # 2. Corrected CatBoost Objective
# # ---------------------------------------------------------
# def objective_catboost(trial):
#     params = {
#         # Search Space
#         'iterations': trial.suggest_int('iterations', 800, 2000), # Increased min iterations
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
#         'depth': trial.suggest_int('depth', 4, 8), # Reduced max depth to save memory
#         'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
#         'random_strength': trial.suggest_float('random_strength', 1e-3, 5.0, log=True),
#         'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        
#         # Fixed GPU Params
#         'loss_function': 'MultiRMSE',
#         'task_type': 'GPU',
#         'boosting_type': 'Plain', 
#         'devices': '0',
#         'verbose': 0,
#         'random_state': 42,
#         'allow_writing_files': False # Prevents creating log files
#     }
    
#     model = CatBoostRegressor(**params)
    
#     # Removed n_splits argument; it now uses whatever is in 'train'
#     return get_cv_score(model, train_siglip_df, feature_engine=feat_engine)

# # ---------------------------------------------------------
# # 3. Corrected XGBoost Objective
# # ---------------------------------------------------------
# def objective_xgboost(trial):
#     params = {
#         'n_estimators': trial.suggest_int('n_estimators', 800, 2000),
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
#         'max_depth': trial.suggest_int('max_depth', 3, 8),
#         'subsample': trial.suggest_float('subsample', 0.6, 1.0),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
#         'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 10.0, log=True),
#         'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 10.0, log=True),
        
#         # Fixed
#         'tree_method': 'hist',
#         'device': 'cuda',
#         'n_jobs': -1,
#         'random_state': 42,
#         'verbosity': 0
#     }
    
#     model = MultiOutputRegressor(XGBRegressor(**params))
#     return get_cv_score(model, train_siglip_df, feature_engine=feat_engine)

# # ---------------------------------------------------------
# # 4. Corrected LightGBM Objective
# # ---------------------------------------------------------
# def objective_lgbm(trial):
#     params = {
#         'n_estimators': trial.suggest_int('n_estimators', 800, 2000),
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
#         'num_leaves': trial.suggest_int('num_leaves', 20, 100),
#         'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
#         'subsample': trial.suggest_float('subsample', 0.6, 1.0),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
#         'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 10.0, log=True),
#         'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 10.0, log=True),
        
#         # Fixed
#         'device': 'gpu',
#         'n_jobs': -1,
#         'random_state': 42,
#         'verbose': -1
#     }
    
#     model = MultiOutputRegressor(LGBMRegressor(**params))
#     return get_cv_score(model, train_siglip_df, feature_engine=feat_engine)

In [ ]:
# # # --- 1. Tune CatBoost (Highest Priority) ---
# # print("Tuning CatBoost...")
# # study_cat = optuna.create_study(direction='maximize')
# # study_cat.optimize(objective_catboost, n_trials=20)

# # print("Best CatBoost Params:", study_cat.best_params)
# # best_cat_params = study_cat.best_params
# # # Re-add fixed params that Optuna didn't tune
# # best_cat_params.update({
# #     'loss_function': 'MultiRMSE', 
# #     # 'task_type': 'GPU', 
# #     'boosting_type': 'Plain', 
# #     # 'devices': '0', 
# #     'verbose': 0, 
# #     'random_state': 42
# # })


# # # --- 2. Tune XGBoost ---
# # print("\nTuning XGBoost...")
# # study_xgb = optuna.create_study(direction='maximize')
# # study_xgb.optimize(objective_xgboost, n_trials=20)

# # print("Best XGBoost Params:", study_xgb.best_params)
# # # best_xgb_params = study_xgb.best_params
# # # best_xgb_params.update({
# # #     'tree_method': 'hist', 
# # #     'device': 'cuda', 
# # #     'n_jobs': -1, 
# # #     'random_state': 42
# # # })


# # --- 3. Tune LightGBM ---
# print("\nTuning LightGBM...")
# study_lgbm = optuna.create_study(direction='maximize')
# study_lgbm.optimize(objective_lgbm, n_trials=20)

# print("Best LightGBM Params:", study_lgbm.best_params)
# # best_lgbm_params = study_lgbm.best_params
# # best_lgbm_params.update({
# #     'device': 'gpu', 
# #     'n_jobs': -1, 
# #     'random_state': 42, 
# #     'verbose': -1
# # })

## Parameters

### Siglip (PCA n_components=0.8)

```
Best CatBoost Params: {'iterations': 1900, 'learning_rate': 0.04488950669764926, 'depth': 4, 'l2_leaf_reg': 0.5647720146150716, 'random_strength': 0.04455012279134044, 'bagging_temperature': 0.9810426313146956}

Best XGBoost Params: {'n_estimators': 1354, 'learning_rate': 0.010266591943008255, 'max_depth': 3, 'subsample': 0.6035540714532827, 'colsample_bytree': 0.9029950550994382, 'reg_alpha': 0.11110779086383878, 'reg_lambda': 9.314996597001533}

Best LightGBM Params: {'n_estimators': 807, 'learning_rate': 0.014069585873331451, 'num_leaves': 48, 'min_child_samples': 19, 'subsample': 0.7451600778259232, 'colsample_bytree': 0.7457374193240649, 'reg_alpha': 0.21580623254415052, 'reg_lambda': 3.784221570035411}
```

### Siglip (NO PCA)

```
Best CatBoost Params: {'iterations': 1945, 'learning_rate': 0.025612792183534742, 'depth': 5, 'l2_leaf_reg': 0.0011451976652037553, 'random_strength': 0.03363171953662423, 'bagging_temperature': 0.9926373709983951}

Best XGBoost Params: {'n_estimators': 1695, 'learning_rate': 0.013048089867977527, 'max_depth': 4, 'subsample': 0.7151550925326732, 'colsample_bytree': 0.7883122143141527, 'reg_alpha': 0.6732457617935534, 'reg_lambda': 8.692842925053135}

Best LightGBM Params: {'n_estimators': 1983, 'learning_rate': 0.03365765614894731, 'num_leaves': 21, 'min_child_samples': 44, 'subsample': 0.9970297203974366, 'colsample_bytree': 0.9324460763054059, 'reg_alpha': 0.324803213211078, 'reg_lambda': 0.1601835613567248}
```

# Multioutput

In [ ]:
# import numpy as np
# import pandas as pd
# from copy import deepcopy
# from sklearn.base import clone
# from sklearn.decomposition import PCA
# from sklearn.multioutput import MultiOutputRegressor
# from sklearn.multioutput import RegressorChain

# # Import the specific libraries
# from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor
# from catboost import CatBoostRegressor

# # ---------------------------------------------------------
# # 1. The Optimized Multi-Output GPU CV Function
# # ---------------------------------------------------------
# def cross_validate_multioutput_gpu(model, train_data, test_data, feature_engine, target_transform='max', n_splits=5):
#     """
#     Performs Cross Validation using a Multi-Output strategy (Vectorized).
    
#     Args:
#         model: An estimator capable of Multi-Output regression (e.g. XGBoost, MultiOutputRegressor)
#         train_data: Training DataFrame
#         test_data: Test DataFrame
#         feature_engine: A transformer (pipeline or class) with .fit() and .transform()
#         target_transform: 'log' (np.log1p), 'max' (TARGET_MAX division), or None
#         n_splits: Number of CV folds
#     """
    
#     # 1. Setup Target Max Array (needed for 'max' scaling and broadcasting)
#     target_max_arr = np.array([TARGET_MAX[t] for t in TARGET_NAMES], dtype=float)
#     y_true = train_data[TARGET_NAMES].values
    
#     # 2. Pre-allocate arrays
#     # We use a zero-filled array for OOF predictions to fill via indexing
#     y_pred = np.zeros([len(train_data), len(TARGET_NAMES)], dtype=float)
#     y_pred_test = np.zeros([len(test_data), len(TARGET_NAMES)], dtype=float)

#     print(f"Starting CV with model: {model.__class__.__name__}")
#     print(f"Target Transform Strategy: {target_transform}")

#     for fold in range(n_splits):
#         # ------------------------------
#         # Data Preparation
#         # ------------------------------
#         # Create masks
#         train_mask = train_data['fold'] != fold
#         valid_mask = train_data['fold'] == fold
#         val_idx = train_data[valid_mask].index

#         # Raw Inputs (Features)
#         X_train_raw = train_data.loc[train_mask, COLUMNS].values
#         X_valid_raw = train_data.loc[valid_mask, COLUMNS].values
#         X_test_raw = test_data[COLUMNS].values
        
#         # Raw Targets
#         y_train = train_data.loc[train_mask, TARGET_NAMES].values
#         # y_valid is used only for metric calculation later, not for training
#         y_valid = train_data.loc[valid_mask, TARGET_NAMES].values

#         # ------------------------------
#         # 1) Transform Targets (Vectorized)
#         # ------------------------------
#         if target_transform == 'log':
#             # Log(1+x) handles zeros and stabilizes variance
#             y_train_proc = np.log1p(y_train)
#         elif target_transform == 'max':
#             # Vectorized division
#             y_train_proc = y_train / target_max_arr
#         else:
#             y_train_proc = y_train
        
#         # ------------------------------
#         # 2) Feature Engineering
#         # ------------------------------
#         # Fit engine only on training data for this fold to avoid leakage
#         engine = deepcopy(feature_engine)
#         engine.fit(X_train_raw)
        
#         x_train_eng = engine.transform(X_train_raw)
#         x_valid_eng = engine.transform(X_valid_raw)
#         x_test_eng = engine.transform(X_test_raw)

#         # ------------------------------
#         # 3) Train (Multi-Output)
#         # ------------------------------
#         # Clone ensures a fresh model instance per fold
#         regr = clone(model) 
        
#         # Fit on (N_samples, N_targets)
#         # Assuming model supports multi-output natively
#         regr.fit(x_train_eng, y_train_proc)

#         # ------------------------------
#         # 4) Predict & Unscale
#         # ------------------------------
#         # Predict returns (N_samples, N_targets)
#         valid_pred_raw = np.array(regr.predict(x_valid_eng))
#         test_pred_raw = np.array(regr.predict(x_test_eng))

#         # Inverse Transform
#         if target_transform == 'log':
#             # Inverse of log1p is expm1
#             valid_pred = np.expm1(valid_pred_raw)
#             test_pred = np.expm1(test_pred_raw)
#         elif target_transform == 'max':
#             # Vectorized multiplication
#             valid_pred = valid_pred_raw * target_max_arr
#             test_pred = test_pred_raw * target_max_arr
#         else:
#             valid_pred = valid_pred_raw
#             test_pred = test_pred_raw

#         # Clip negative predictions (Physical impossibility for biomass/yield)
#         valid_pred = valid_pred.clip(0)
#         test_pred = test_pred.clip(0)

#         # Store OOF
#         y_pred[val_idx] = valid_pred
        
#         # Accumulate Test Preds
#         y_pred_test += test_pred / n_splits

#         # Optional: Print fold score if metric function exists
#         try:
#             fold_score = competition_metric(y_valid, valid_pred)
#             print(f"  Fold {fold} score: {fold_score:.6f}")
#         except NameError:
#             pass # competition_metric not defined
            
#         if fold == 0:
#              print(f"  [Fold 0 Debug] Transformed Train Shape: {x_train_eng.shape}")

#     # Global CV score
#     try:
#         full_cv = competition_metric(y_true, y_pred)
#         print(f"Full CV Score: {full_cv:.6f}")
#     except NameError:
#         print("Done (metric function not found)")

#     return y_pred, y_pred_test

# def cross_validate_regressor_chain(model, train_data, test_data, feature_engine, target_transform='max', chain_order=None, n_splits=5, random_state=42):
#     """
#     Performs Cross Validation using a Regressor Chain strategy.
    
#     RegressorChain fits a separate model for each target, but uses the predictions 
#     of previous targets in the chain as features for subsequent targets.
    
#     Args:
#         model: A base estimator (e.g. XGBoost, Ridge). NOT a MultiOutputRegressor.
#         train_data: Training DataFrame
#         test_data: Test DataFrame
#         feature_engine: A transformer (pipeline or class) with .fit() and .transform()
#         target_transform: 'log', 'max', or None
#         chain_order: Order of targets. None (default order), 'random', or list of indices.
#         n_splits: Number of CV folds
#     """
    
#     # 1. Setup Target Max Array
#     target_max_arr = np.array([TARGET_MAX[t] for t in TARGET_NAMES], dtype=float)
#     y_true = train_data[TARGET_NAMES].values
    
#     # 2. Pre-allocate arrays
#     y_pred = np.zeros([len(train_data), len(TARGET_NAMES)], dtype=float)
#     y_pred_test = np.zeros([len(test_data), len(TARGET_NAMES)], dtype=float)

#     print(f"Starting Chain CV with base model: {model.__class__.__name__}")
#     print(f"Target Transform: {target_transform} | Chain Order: {chain_order}")

#     for fold in range(n_splits):
#         # ------------------------------
#         # Data Preparation
#         # ------------------------------
#         train_mask = train_data['fold'] != fold
#         valid_mask = train_data['fold'] == fold
#         val_idx = train_data[valid_mask].index

#         # Raw Inputs
#         X_train_raw = train_data.loc[train_mask, COLUMNS].values
#         X_valid_raw = train_data.loc[valid_mask, COLUMNS].values
#         X_test_raw = test_data[COLUMNS].values
        
#         # Raw Targets
#         y_train = train_data.loc[train_mask, TARGET_NAMES].values
#         y_valid = train_data.loc[valid_mask, TARGET_NAMES].values

#         # ------------------------------
#         # 1) Transform Targets
#         # ------------------------------
#         if target_transform == 'log':
#             y_train_proc = np.log1p(y_train)
#         elif target_transform == 'max':
#             y_train_proc = y_train / target_max_arr
#         else:
#             y_train_proc = y_train
        
#         # ------------------------------
#         # 2) Feature Engineering
#         # ------------------------------
#         engine = deepcopy(feature_engine)
#         engine.fit(X_train_raw)
        
#         x_train_eng = engine.transform(X_train_raw)
#         x_valid_eng = engine.transform(X_valid_raw)
#         x_test_eng = engine.transform(X_test_raw)

#         # ------------------------------
#         # 3) Train (Regressor Chain)
#         # ------------------------------
#         # We clone the base model to ensure a fresh start
#         base_model = clone(model)
        
#         # Create the Chain
#         # Note: If chain_order is 'random', we need the random_state to be consistent
#         chain = RegressorChain(base_estimator=base_model, order=chain_order, random_state=random_state)
        
#         # Fit the chain on processed targets
#         # Internally, this fits Model 1, then adds Model 1's preds to X to fit Model 2, etc.
#         chain.fit(x_train_eng, y_train_proc)

#         # ------------------------------
#         # 4) Predict & Unscale
#         # ------------------------------
#         # Chain.predict automatically handles the feed-forward of predictions
#         valid_pred_raw = np.array(chain.predict(x_valid_eng))
#         test_pred_raw = np.array(chain.predict(x_test_eng))

#         # Inverse Transform
#         if target_transform == 'log':
#             valid_pred = np.expm1(valid_pred_raw)
#             test_pred = np.expm1(test_pred_raw)
#         elif target_transform == 'max':
#             valid_pred = valid_pred_raw * target_max_arr
#             test_pred = test_pred_raw * target_max_arr
#         else:
#             valid_pred = valid_pred_raw
#             test_pred = test_pred_raw

#         # Clip negative predictions
#         valid_pred = valid_pred.clip(0)
#         test_pred = test_pred.clip(0)

#         # Store OOF
#         y_pred[val_idx] = valid_pred
#         y_pred_test += test_pred / n_splits

#         # Optional: Print fold score
#         try:
#             fold_score = competition_metric(y_valid, valid_pred)
#             print(f"  Fold {fold} score: {fold_score:.6f}")
#         except NameError:
#             pass

#         if fold == 0:
#              print(f"  [Fold 0 Debug] Train Shape: {x_train_eng.shape}")

#     # Global CV score
#     try:
#         full_cv = competition_metric(y_true, y_pred)
#         print(f"Full CV Score: {full_cv:.6f}")
#     except NameError:
#         print("Done (metric function not found)")

#     return y_pred, y_pred_test

# feat_engine = EmbeddingFeatureEngine(
#     n_pca_components=0.90, 
#     n_clusters=25, 
#     use_stats=True, 
#     use_similarity=True,
#     use_anomaly=True,        # Adds Anomaly Score
#     use_entropy=True,        # Adds Entropy
#     use_pca_interactions=True # Adds Poly features on Top 5 PCA
# )

# # ---------------------------------------------------------
# # 2. Model Definitions
# # ---------------------------------------------------------
# best_cat_params = {
#     'iterations': 1783, 
#     'learning_rate': 0.0633221588945314, 
#     'depth': 4, 
#     'l2_leaf_reg': 0.1312214556803292, 
#     'random_strength': 0.04403178418151252, 
#     'bagging_temperature': 0.9555074383215754
# }
# best_cat_params.update({
#     'loss_function': 'MultiRMSE', 
#     # 'task_type': 'GPU', 
#     'boosting_type': 'Plain', 
#     # 'devices': '0', 
#     'verbose': 0, 
#     'random_state': 42
# })

# best_xgb_params = {
#     'n_estimators': 1501, 
#     'learning_rate': 0.024461148923117938, 
#     'max_depth': 3, 
#     'subsample': 0.6905614627569726, 
#     'colsample_bytree': 0.895428293256401, 
#     'reg_alpha': 0.4865138988842402, 
#     'reg_lambda': 0.6015849227570268
# }
# best_xgb_params.update({
#     'tree_method': 'hist', 
#     # 'device': 'cuda', 
#     'n_jobs': -1, 
#     'random_state': 42
# })

# best_lgbm_params = {
#     'n_estimators': 1232, 
#     'learning_rate': 0.045467475791811464, 
#     'num_leaves': 32, 
#     'min_child_samples': 38, 
#     'subsample': 0.9389508238313968, 
#     'colsample_bytree': 0.8358504077200445, 
#     'reg_alpha': 0.10126277169074206, 
#     'reg_lambda': 0.1357065010990351
# }
# best_lgbm_params.update({
#     # 'device': 'gpu', 
#     'n_jobs': -1, 
#     'random_state': 42, 
#     'verbose': -1
# })

# # --- A. XGBoost (Wrapped) ---
# # XGBoost requires MultiOutputRegressor wrapper for multi-target
# xgb_model = MultiOutputRegressor(
#     XGBRegressor(
#         **best_xgb_params
#     )
# )

# # --- B. LightGBM (Wrapped) ---
# # LightGBM requires MultiOutputRegressor wrapper.
# # Note: Ensure you have the GPU-compiled version of LightGBM installed.
# lgbm_model = MultiOutputRegressor(
#     LGBMRegressor(
#         **best_lgbm_params
#     )
# )

# # --- C. CatBoost (Native) ---
# # CatBoost supports "MultiRMSE" natively. No wrapper needed.
# # This is usually the fastest option for multi-target on GPU.
# cat_model = CatBoostRegressor(
#     **best_cat_params
# )

# # ---------------------------------------------------------
# # 3. Usage Example
# # ---------------------------------------------------------
# # Assuming 'train' and 'test' pandas DataFrames exist
# # and TARGET_NAMES / TARGET_MAX / COLUMNS are defined globally

# # 1. Run XGBoost
# print("\n--- Running XGBoost ---")
# oof_xgb, test_xgb = cross_validate_multioutput_gpu(xgb_model, train_siglip_df, test_siglip_df, feature_engine=feat_engine)
# compare_results(oof_xgb, train_siglip_df)

# # 2. Run LightGBM
# print("\n--- Running LightGBM ---")
# oof_lgbm, test_lgbm = cross_validate_multioutput_gpu(lgbm_model, train_siglip_df, test_siglip_df, feature_engine=feat_engine)
# compare_results(oof_lgbm, train_siglip_df)

# # 3. Run CatBoost
# print("\n--- Running CatBoost ---")
# oof_cat, test_cat = cross_validate_multioutput_gpu(cat_model, train_siglip_df, test_siglip_df, feature_engine=feat_engine)
# compare_results(oof_cat, train_siglip_df)

# print("\n######## Ridge Regression #######")
# # ridge_model = MultiOutputRegressor(
# #     Ridge()
# # )
# oof_ridge, pred_test_ri = cross_validate_multioutput_gpu(
#     Ridge(), 
#     train_siglip_df, test_siglip_df, 
#     feature_engine=feat_engine,
# )
# compare_results(oof_ridge, train_siglip_df)

# print("\n###### Bayesian Ridge Regressor #######")
# bayesian_model = MultiOutputRegressor(
#     BayesianRidge()
# )
# oof_bayesian, pred_test_bri = cross_validate_multioutput_gpu(
#     bayesian_model, 
#     train_siglip_df, test_siglip_df, 
#     feature_engine=feat_engine,
# )
# compare_results(oof_bayesian, train_siglip_df)

# # print("\n###### Hist Gradient Boosting Regressor ######")
# # hist_model = MultiOutputRegressor(
# #     HistGradientBoostingRegressor()
# # )
# # _, pred_test_hb = cross_validate_regressor_chain(
# #     HistGradientBoostingRegressor(), 
# #     train_siglip_df, test_siglip_df, 
# #     feature_engine=feat_engine,
# #     chain_order=chain_order_indices,   # <--- PASS THE ORDER HERE
# # )

# # print("\n##### Extra Trees Regressor ######")
# # et_model = MultiOutputRegressor(
# #     ExtraTreesRegressor()
# # )
# # _, pred_test_et = cross_validate_regressor_chain(
# #     ExtraTreesRegressor(), 
# #     train_siglip_df, test_siglip_df, 
# #     feature_engine=feat_engine
# # )

In [ ]:
pred_test = (
    pred_test_gb
    + pred_test_hb
    # + pred_test_svr
    # + pred_test_et
) / 2

# pred_test = (
#     pred_test_hb
#     + pred_test_et
# ) / 2

# pred_test = (
#     pred_test_gb
#     + pred_test_hb
#     + pred_test_et
# ) / 3

# pred_test = (
#     pred_test_ri
#     + pred_test_gb
#     + pred_test_hb
#     + pred_test_et
# ) / 4

# pred_test = (test_xgb + test_lgbm + test_cat + pred_test_ri + pred_test_bri) / 5
# pred_test = 0.6*pred_test_ri + 0.15*pred_test_gb + 0.15*pred_test_hb + 0.1*pred_test_et

In [ ]:
test_df[TARGET_NAMES] = pred_test
test_df = post_process_biomass(test_df)
# test_df['GDM_g'] = test_df['Dry_Green_g'] + test_df['Dry_Clover_g']
# test_df['Dry_Total_g'] = test_df['GDM_g'] + test_df['Dry_Dead_g']
sub_df = melt_table(test_df)
sub_df[['sample_id', 'target']].to_csv("submission.csv", index=False)

In [ ]:
pd.read_csv("submission.csv")